# Doc2Vecの実行条件調査（part3）

- ラベルごとの教師データを最大１０件とします（このためテストデータから学習セットの大半が失われます）


- 品詞を落とさないようにする


- パラメータを一部変更（DBOWを使用／feature数を調整）


accuracy=0.945 とややダウンしました。

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

## (2) Doc2Vecの動作確認

### (2-1) コーパス生成

コーパス（単語が半角スペースで区切られた文字列）生成時、一部の品詞を落とすようにします。

（＝learning.core.nlang.Nlang クラスの仕様に従います）

In [2]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 9  # bot_id = 9はセプテーニ
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}

learning_parameter = LearningParameter(attr)

In [3]:
_datasource = Datasource(type='csv')
learning_training_messages = _datasource.learning_training_messages(_bot_id)
questions = np.array(learning_training_messages['question'])
answer_ids = np.array(learning_training_messages['answer_id'])

2017/05/18 PM 01:55:25 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/18 PM 01:55:25 ['./fixtures/question_answers/toyotsu_human.csv']


In [4]:
import MeCab
import mojimoji

class Nlang_naive:
    @classmethod
    def split(self, text):
        tagger = MeCab.Tagger("-u learning/dict/custom.dic")
        tagger.parse('')  # node.surfaceを取得出来るようにするため、空文字をparseする(Python3のバグの模様)
        node = tagger.parseToNode(text)
        word_list = []
        while node:
            features = node.feature.split(",")
            pos = features[0]
            if pos in ["BOS/EOS", "記号"]:
                node = node.next
                continue

            #print(features)
            lemma = node.feature.split(",")[6]

            if lemma == "*":
                lemma = node.surface  #.decode("utf-8")
                
            word_list.append(mojimoji.han_to_zen(lemma))
            node = node.next
        return " ".join(word_list)

    @classmethod
    def batch_split(self, texts):
        splited_texts = []
        for text in texts:
            splited_texts.append(self.split(text))
        return splited_texts

In [5]:
from learning.core.nlang import Nlang

_sentences = np.array(questions)
_separated_sentences = Nlang_naive.batch_split(_sentences)

#### ラベル毎の教師データ数を制限します。

In [6]:
answer_id_count = {}

selected_answer_ids = []
selected_questions = []
selected_separated_sentences = []

sample_limit_count = 10

for index, answer_id in enumerate(answer_ids):
    if answer_id not in answer_id_count.keys():
        answer_id_count[answer_id] = 0

    if answer_id_count[answer_id] < sample_limit_count:
        '''
            データをリストに格納
        '''
        selected_answer_ids.append('%04d_%02d' % (answer_id, answer_id_count[answer_id]))
        selected_questions.append(questions[index])
        selected_separated_sentences.append(_separated_sentences[index])
        answer_id_count[answer_id] += 1

### (2-2) コーパスにタグ付け

models.doc2vecの仕様に従います。

In [7]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

def doc_to_sentence(sentences, name):
    words = sentences.split(' ')
    return TaggedDocument(words=words, tags=[name])

def corpus_to_sentences(separated_sentences, answer_ids):
    for idx, (doc, name) in enumerate(zip(separated_sentences, answer_ids)):
        yield doc_to_sentence(doc, name)

### (2-3) 学習処理／モデルのシリアライズ

In [8]:
'''
    ユニークになったコーパス／ラベルから、
    学習セットを生成する
    
    教師データ件数＝1730
'''
sentences = corpus_to_sentences(selected_separated_sentences, selected_answer_ids)
sentence_list = list(sentences)
len(sentence_list)

1730

In [9]:
'''
    パラメータ変更点：
    　学習時にDBOWを使用する
    　featureの数をサンプル件数に近い値に設定
    　反復回数をfeature数の10倍に設定
'''
n_feature = round(len(sentence_list) + 49, -2) # 切り上げし、100の倍数に変更
n_iter = n_feature * 10
print('Doc2Vec: feature(size)=%d, iter=%d' % (n_feature, n_iter))

model = Doc2Vec(dm=0, size=n_feature, min_count=1, iter=n_iter)

Doc2Vec: feature(size)=1800, iter=18000


In [10]:
'''
    ボキャブラリ生成／学習実行
    学習モデルは、ファイルに保存しておく
'''
model.build_vocab(sentence_list)
model.train(sentence_list)

model_path = 'prototype/better_algorithm/doc2vec.model'
model.save(model_path)

In [11]:
'''
    モデル内に保持されているベクトルの数を取得
    （教師データ数と同じであることを確認）
'''
len(model.docvecs)

1730

In [12]:
model.docvecs

### (2-4) 予測処理

質問文は一部「揺れ」を与えたものとなっております

In [13]:
model_path = 'prototype/better_algorithm/doc2vec.model'

def predict(word, model_path):
    '''
        予測処理にかけるコーパスを生成
        （学習セット作成時と同じ関数を使用）
    '''
    corpus = Nlang_naive.split(word).split()

    '''
        コーパスからベクトルを生成し、
        ロードしたモデルから類似ベクトルを検索
    '''
    loaded_model = models.Doc2Vec.load(model_path)
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    return corpus, ret

In [14]:
'''
    マウスが破損（正解＝4458）
'''
predict('マウスが破損', model_path)

(['マウス', 'が', '破損'],
 [('4458_00', 0.8120925426483154),
  ('4581_01', 0.6716203093528748),
  ('4480_07', 0.6600103378295898),
  ('4480_04', 0.6563659906387329),
  ('4480_01', 0.6553093791007996),
  ('4564_02', 0.6523987650871277),
  ('4581_03', 0.6489686965942383),
  ('4480_08', 0.6426993608474731),
  ('4458_02', 0.6380584239959717),
  ('4458_04', 0.6376777291297913)])

In [15]:
'''
    無線を使用したい（正解＝4516）
'''
predict('無線を使用したい', model_path)

(['無線', 'を', '使用', 'する', 'たい'],
 [('4516_00', 0.8298698663711548),
  ('4521_00', 0.7635502219200134),
  ('4577_04', 0.7563982009887695),
  ('4557_07', 0.7358416318893433),
  ('4625_03', 0.7092441320419312),
  ('4625_08', 0.7077388167381287),
  ('4429_00', 0.6877231597900391),
  ('4592_00', 0.6870255470275879),
  ('4600_01', 0.6853108406066895),
  ('4447_08', 0.6651139259338379)])

In [16]:
'''
    情報システムのアドレス（正解＝7040）
'''
predict('情報システムのアドレス', model_path)

(['情報', 'システム', 'の', 'アドレス'],
 [('7040_01', 0.8174411654472351),
  ('7040_04', 0.7799542546272278),
  ('7040_02', 0.697982132434845),
  ('4547_03', 0.6650873422622681),
  ('4547_05', 0.6641379594802856),
  ('4547_01', 0.6559579372406006),
  ('4514_01', 0.6520905494689941),
  ('4508_00', 0.613314151763916),
  ('4492_09', 0.6111562848091125),
  ('4621_08', 0.6042506098747253)])

In [17]:
'''
    Office2010を使用したいのですが（正解＝4625）
'''
predict('Office2010を使用したいのですが', model_path)

(['Ｏｆｆｉｃｅ', '２０１０', 'を', '使用', 'する', 'たい', 'の', 'です', 'が'],
 [('4625_08', 0.7364158630371094),
  ('4625_03', 0.7302572727203369),
  ('4625_01', 0.7112269401550293),
  ('4523_02', 0.7084237337112427),
  ('4523_03', 0.6895222663879395),
  ('4625_02', 0.6819004416465759),
  ('4625_04', 0.6452490091323853),
  ('4625_00', 0.6394310593605042),
  ('4523_06', 0.6384126543998718),
  ('4523_09', 0.6342065930366516)])

In [18]:
'''
    携帯からサイボウズを使いたいのですが、どうしたら出来ますか？
    （正解＝4504だが、学習時の教師データから抜け落ちたもの）
'''
predict('携帯からサイボウズを使いたいのですが、どうしたら出来ますか？', model_path)

(['携帯',
  'から',
  'サイボウズ',
  'を',
  '使う',
  'たい',
  'の',
  'です',
  'が',
  'どう',
  'する',
  'た',
  '出来る',
  'ます',
  'か'],
 [('4541_07', 0.7306663393974304),
  ('4501_06', 0.7160975337028503),
  ('4557_09', 0.604280948638916),
  ('4541_05', 0.5856301188468933),
  ('4501_00', 0.5848420858383179),
  ('4541_04', 0.5771788954734802),
  ('4541_06', 0.5708301067352295),
  ('4501_07', 0.5668818950653076),
  ('4541_00', 0.5627776384353638),
  ('4504_04', 0.5417144894599915)])

## (3) accuracy 測定

ここからが本題です

In [19]:
model_path = 'prototype/better_algorithm/doc2vec.model'

len(selected_separated_sentences)

1730

In [20]:
def predict_similarity(separated_sentence, model_path):
    corpus = separated_sentence.split()
    loaded_model = models.Doc2Vec.load(model_path)
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    answer_id, similarity = ret[0]
    return corpus, answer_id, similarity

In [21]:
def get_prediction_statistics(separated_sentences, answer_ids, model_path):
    '''
        学習セットの質問文をそのまま予測処理にかけて、
        回答を予測
    '''
    statistics = []
    for i, _ in enumerate(separated_sentences):
        sentence = separated_sentences[i]
        preferred_answer_id = answer_ids[i]
        corpus, answer_id, similarity = predict_similarity(separated_sentences[i], model_path)
        corpus_len = len(corpus)
        statistics.append((i, corpus_len, preferred_answer_id, answer_id, similarity))

    return statistics

In [22]:
prediction_statistics = get_prediction_statistics(selected_separated_sentences, selected_answer_ids, model_path)

In [23]:
ncorrect_by_corpus_len = {}
nsample_by_corpus_len = {}

ncorrect = 0
nsample = 0

'''
    予測結果を、質問文の単語数毎／回答ID毎に統計する
'''
for statistics in prediction_statistics:
    i, corpus_len, preferred_answer_id, answer_id, similarity = statistics
    
    '''
        質問文の単語数ごとに統計を取る
    '''
    if corpus_len not in nsample_by_corpus_len.keys():
        ncorrect_by_corpus_len[corpus_len] = 0
        nsample_by_corpus_len[corpus_len] = 0
    nsample_by_corpus_len[corpus_len] += 1
    
    '''
        正解かどうか検査
        （NNNN_nn 形式ラベルの上４桁が一致していれば正解とします）
    '''
    nsample += 1
    if preferred_answer_id[0:4] == answer_id[0:4]:
        ncorrect += 1
        ncorrect_by_corpus_len[corpus_len] += 1

In [24]:
'''
    質問文の単語数ごとの統計情報を編集
'''
info_by_corpus_len = []
for k, v in ncorrect_by_corpus_len.items():
    info_by_corpus_len.append((
        k, 
        ncorrect_by_corpus_len[k]/nsample_by_corpus_len[k], 
        ncorrect_by_corpus_len[k], 
        nsample_by_corpus_len[k]
    ))

In [25]:
'''
    全体の正解率
'''
print("accuracy=%0.3f (%d/%d)" % (
    ncorrect/nsample, ncorrect, nsample))

accuracy=0.945 (1634/1730)


In [26]:
'''
    質問文の単語数ごとの正解率をリスト
'''
for info in info_by_corpus_len:
    print("word_count=%2d: accuracy=%0.3f (%d/%d)" % (
        info[0], info[1], info[2], info[3]
    ))

word_count= 1: accuracy=1.000 (3/3)
word_count= 2: accuracy=0.964 (108/112)
word_count= 3: accuracy=0.922 (236/256)
word_count= 4: accuracy=0.898 (265/295)
word_count= 5: accuracy=0.919 (192/209)
word_count= 6: accuracy=0.944 (151/160)
word_count= 7: accuracy=0.940 (125/133)
word_count= 8: accuracy=0.946 (106/112)
word_count= 9: accuracy=1.000 (69/69)
word_count=10: accuracy=0.986 (71/72)
word_count=11: accuracy=1.000 (60/60)
word_count=12: accuracy=1.000 (49/49)
word_count=13: accuracy=1.000 (56/56)
word_count=14: accuracy=1.000 (40/40)
word_count=15: accuracy=0.966 (28/29)
word_count=16: accuracy=1.000 (23/23)
word_count=17: accuracy=1.000 (14/14)
word_count=18: accuracy=1.000 (6/6)
word_count=19: accuracy=1.000 (4/4)
word_count=20: accuracy=1.000 (9/9)
word_count=21: accuracy=1.000 (4/4)
word_count=22: accuracy=1.000 (3/3)
word_count=23: accuracy=1.000 (1/1)
word_count=24: accuracy=1.000 (1/1)
word_count=27: accuracy=1.000 (1/1)
word_count=29: accuracy=1.000 (2/2)
word_count=30: acc